In [1]:
from neural_network import NeuralNetwork
from sgd import SGD
from quickprop import Quickprop
from rprop import Rprop
from lr_schedulers import ExponentialDecayScheduler, TimeBasedScheduler, StepDecayScheduler
from grid_search import GridSearch, RandomSearch
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid

In [2]:
#MONK PROBLEMS
def load_monks_problems():
    monks_trains={}
    monks_tr_targets={}

    monks_tests={}
    monks_ts_targets={}

    for i in ["1","2","3"]:
        path="../monks/monks"+i;
        train_set = np.genfromtxt(path+"-train.txt", delimiter=" ", dtype="int")[:,1:-1] #rimuove la colonna nan alla fine
        train_targets = np.genfromtxt(path+"-train.txt", delimiter=" ", dtype="int")[:,:1]
        monks_trains["monks"+i]=train_set
        monks_tr_targets["monks"+i]=train_targets

        test_set = np.genfromtxt(path+"-test.txt", delimiter=" ", dtype="int") [:,1:-1] #rimuove targets e colonna nan 
        test_targets = np.genfromtxt(path+"-test.txt", delimiter=" ", dtype="int") [:,0:1]
        monks_tests["monks"+i]=test_set
        monks_ts_targets["monks"+i]=test_targets
        
    return monks_trains, monks_tr_targets, monks_tests, monks_ts_targets

In [3]:
monks_trains, monks_tr_targets, monks_tests, monks_ts_targets = load_monks_problems()

In [4]:
#It creates the nn and starts grid searching
#Returns a list of tuples grid_results=[(accuracy, params, nn_layers)]

def compose_nn(dataset, targets, params, debug=False):
    grid_results=[]
    max_hidden_layers=5
    num_of_units=[5,10,15]

    for i_hidden_layers in range(1,max_hidden_layers, 1):
        for j_units in num_of_units:
            nn = NeuralNetwork()

            nn.add_layer('dense', j_units, 'sigmoid', dataset.shape[1])
            for h in range(2,i_hidden_layers+1,1):
                nn.add_layer('dense', j_units, 'sigmoid')
            nn.add_layer('dense', 1, 'sigmoid')
            
            gs = GridSearch(params, nn)
            
            l=[]
            for layer in nn.layers:
                l.append(layer.num_units)
            
            grid_results.append(tuple((gs.fit(dataset, targets), l)))
            
            if(debug==True):
                print(">layers: (", len(nn.layers),")", l)
                print(grid_results[-1])
    return grid_results

In [5]:
# --- parameters ---
param_grid = {
    'lr': [0.05, 0.1, 0.15, 0.2], 
    'epoch' : [1000, 2000, 3000],
    'patience': [100, 200, 300],
    'test_size': [0.25, 0.5],
    'batch_size': [16],
    
    'momentum': [0.7, 0.8, 0.9],
    'nesterov': [False]
}

In [6]:
#TEST WITH SMALL PARAM_GRID





#REMOVE IT 






#DON'T FORGET


param_grid = {
    'lr': [0.15, 0.2], 
    'epoch' : [4000],
    'patience': [200],
    'test_size': [0.25, 0.5],
    'batch_size': [16],
    
    'momentum': [0.7, 0.8],
    'nesterov': [False]
}

In [ ]:
#start grids and save all results
results={}
which_monks=["1","2","3"]

for i in which_monks:
    print("monks"+i)
    x=monks_trains["monks"+i]
    y=monks_tr_targets["monks"+i]
    results["monks"+i]=compose_nn(x, y, param_grid, True)

monks1
>layers: ( 2 ) [5, 1]
((0.8387096774193549, {'batch_size': 16, 'epoch': 4000, 'lr': 0.15, 'momentum': 0.7, 'nesterov': False, 'patience': 200, 'test_size': 0.25}), [5, 1])
>layers: ( 2 ) [10, 1]
((0.9354838709677419, {'batch_size': 16, 'epoch': 4000, 'lr': 0.15, 'momentum': 0.7, 'nesterov': False, 'patience': 200, 'test_size': 0.25}), [10, 1])
>layers: ( 2 ) [15, 1]
((0.9838709677419355, {'batch_size': 16, 'epoch': 4000, 'lr': 0.2, 'momentum': 0.8, 'nesterov': False, 'patience': 200, 'test_size': 0.5}), [15, 1])
>layers: ( 3 ) [5, 5, 1]
((0.8709677419354839, {'batch_size': 16, 'epoch': 4000, 'lr': 0.15, 'momentum': 0.7, 'nesterov': False, 'patience': 200, 'test_size': 0.25}), [5, 5, 1])
>layers: ( 3 ) [10, 10, 1]
((1.0, {'batch_size': 16, 'epoch': 4000, 'lr': 0.2, 'momentum': 0.8, 'nesterov': False, 'patience': 200, 'test_size': 0.25}), [10, 10, 1])
>layers: ( 3 ) [15, 15, 1]
((0.967741935483871, {'batch_size': 16, 'epoch': 4000, 'lr': 0.15, 'momentum': 0.7, 'nesterov': False, '

In [75]:
#get best results
best_results={}

for m in results:
    l=results[m]
    best=max(l, key=lambda x: x[0])
    best_results[m]=best

best_results

{'monks1': ((1.0,
   {'batch_size': 16,
    'epoch': 4000,
    'lr': 0.15,
    'momentum': 0.8,
    'nesterov': False,
    'patience': 200,
    'test_size': 0.25}),
  [10, 1]),
 'monks2': ((0.7209302325581395,
   {'batch_size': 16,
    'epoch': 4000,
    'lr': 0.15,
    'momentum': 0.8,
    'nesterov': False,
    'patience': 200,
    'test_size': 0.25}),
  [5, 1]),
 'monks3': ((1.0,
   {'batch_size': 16,
    'epoch': 4000,
    'lr': 0.15,
    'momentum': 0.8,
    'nesterov': False,
    'patience': 200,
    'test_size': 0.25}),
  [10, 10, 10, 1])}

In [7]:
'''
OLD CODE
# --- Parameters grid search ---
def params_grid_search(nn, dataset, target, params):
    best_accuracy=0
    grid = ParameterGrid(param_grid)

    for params in grid:
        nn.compile(optimizer=SGD(lr_init=params['lr'], momentum=params['momentum'], nesterov=False, lr_sched=StepDecayScheduler(epochs_drop=15)))
        model=nn.fit(dataset, target, batch_size=params['batch_size'], test_size=params['test_size'], epochs=params['epoch'], patience=params['patience'], save_stats=True) #lr=0.05 test_size=0.5 epochs=2000

        accuracy=np.amax(np.load("vl_accuracy.npy"))
        if(accuracy > best_accuracy):
            best_accuracy=accuracy
            best_param={x:params[x] for x in list(param_grid.keys())}
            
    return best_accuracy, best_param

# --- Layer/Unit pseudo grid search ---
def grid_search(dataset, target, params, debug=False):
    grid_results=[];
    max_hidden_layers=5
    num_of_units=[5,10,15]

    for i_hidden_layers in range(1,max_hidden_layers, 1):
        for j_units in [5,10,15]:
            nn = NeuralNetwork()

            nn.add_layer('dense', j_units, 'sigmoid', dataset.shape[1])
            for h in range(2,i_hidden_layers+1,1):
                nn.add_layer('dense', j_units, 'sigmoid')
            nn.add_layer('dense', 1, 'sigmoid')
            
            grid_results.append(params_grid_search(nn, dataset, target, params))
            if(debug==True):
                l=[]
                for layer in nn.layers:
                    l.append(layer.num_units)
                print(">layers: (", len(nn.layers),")", l)
                print(grid_results[-1])
    return grid_results
'''